In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import xmltodict, json
import ast
import numbers
import shlex # package to construct the git command to subprocess format
import subprocess 
import os
%matplotlib inline

In [2]:
MasterFile = 'C:\GitHubRepos\ApsimX\Tests\Validation\Wheat\Wheat.apsimx'

In [15]:
def findModel(Parent,modelPath):
    PathElements = modelPath.split('.')
    return findModelFromElements(Parent,PathElements)

def findModelFromElements(Parent,PathElements):
    for pe in PathElements:
        Parent = findNextChild(Parent,pe)
    return Parent

def findNextChild(Parent,ChildName):
    if len(Parent['Children']) >0:
        for child in range(len(Parent['Children'])):
            if Parent['Children'][child]['Name'] == ChildName:
                return Parent['Children'][child]
    else:
        return Parent[ChildName]

def swapModels(Parent,modelName,modelType,NDVIModel):
    pos=0
    for c in Parent['Children']:
        if (c['Name'] == modelName) and (c['$type'] == modelType):
            params = {c["Parameters"][0]['Key']:float(c["Parameters"][0]['Value']),
                     c["Parameters"][1]['Key']:float(c["Parameters"][1]['Value']),
                     c["Parameters"][2]['Key']:float(c["Parameters"][2]['Value']),
                     c["Parameters"][3]['Key']:float(c["Parameters"][3]['Value'])}
            NewModel = NDVIModel.copy()
            NewModel["DrySoilNDVI"] = params["DrySoilNDVI"]
            NewModel["WetSoilNDVI"] = params["WetSoilNDVI"]
            NewModel["GreenCropNDVI"] = params["GreenCropNDVI"]
            NewModel["DeadCropNDVI"]  = params["DeadCropNDVI"] 
            print(str(NewModel["DrySoilNDVI"]))
            Parent['Children'].append(NewModel)
            del Parent['Children'][pos]            
        swapModels(c,modelName,modelType,NDVIModel)
        pos+=1

In [16]:
## Read wheat test file into json object
with open(MasterFile,'r') as MasterJSON:
    Master = json.load(MasterJSON)
    MasterJSON.close()

NDVIModel = findModel(Master,'Validation.New Zealand.Lincoln2024.Lincoln24Base.Field.NDVI')
swapModels(Master,"NDVIModel","Models.Manager, Models",NDVIModel)

with open(MasterFile,'w') as WheatTestsJSON:
    json.dump(Master ,WheatTestsJSON,indent=2)

0.12
0.16
0.16
0.11
0.11
0.13
0.13
0.22
0.22
0.22
0.14
0.14
0.14
0.14
0.14
0.14
0.14
0.14
0.14
0.14
0.14
0.14
0.14
0.14
0.1
0.1
0.1
0.1
0.1
0.1
0.1
0.22
0.22
0.18
0.18
0.18
0.18


In [8]:
NDVIModel

{'$type': 'Models.Sensing.NDVI, Models',
 'DrySoilNDVI': 0.18,
 'WetSoilNDVI': 0.2,
 'GreenCropNDVI': 0.95,
 'DeadCropNDVI': 0.1,
 'Value': 0.0,
 'Name': 'NDVI',
 'ResourceName': None,
 'Children': [],
 'Enabled': True,
 'ReadOnly': False}

In [45]:
#Copy prototype wheat model out of replacements and put it in replacements in test file
NDVIModel =  findModel(Master,'Validation.New Zealand.Lincoln2024.Lincoln24Base.Field.NDVI')
addModel(Master,'Replacements',NewModel)
#Put updated leaf size calculation script into test file
NewModel =  findModel(Prototype,'Replacements.OutputMaxLeafSize')
replaceModel(Master,'Replacements.MaxLeafSize',NewModel)
#Put updated leaf size report into test file
NewModel =  findModel(Prototype,'Replacements.ReportMaxLeafSize')
replaceModel(Master,'Replacements.MaxLeafSize',NewModel)
#bring cultivar descriptions from master back into replacement wheat model
NewModel = findModel(Released,'Wheat.Cultivars')
replaceModel(Master,'Replacements.Wheat.Cultivars',NewModel)
#rename manager scripts to capture max leaf size
renameModelofType(Master,'MaxLeafSize',"Models.Manager, Models",'OutputMaxLeafSize')

In [46]:
os.remove(ImplementedFile)
with open(ImplementedFile,'w') as ImplementedJSON:
    json.dump(Master ,ImplementedJSON,indent=2)

In [47]:
replacements = pd.read_excel(VariableRenamesFile,index_col=0,sheet_name = 'SimpleLeafRenames').to_dict()['SimpleLeaf']
with open(ImplementedFile, 'r') as file: 
    data = file.read() 
    for v in replacements.keys():
        data = data.replace(v, replacements[v])
        w = v.replace('Wheat','[Wheat]')
        rw = replacements[v].replace('Wheat','[Wheat]')
        data = data.replace(w, rw)
        
# Opening our text file in write only 
# mode to write the replaced content 
with open(ImplementedFile, 'w') as file: 
  
    # Writing the replaced data in our 
    # text file 
    file.write(data) 

In [50]:
VariableRenames = pd.read_excel(VariableRenamesFile,index_col=0, sheet_name='SimpleLeafRenames').to_dict()['SimpleLeaf']
MaxLeafSizeRenames = pd.read_excel(VariableRenamesFile,index_col=0, sheet_name='MaxLeafSizeRenames').to_dict()['SimpleLeaf']

from pathlib import Path
fileLoc = 'C:\GitHubRepos\ApsimX\Tests\Validation\Wheat\data'
Allcols = []
pathlist = Path(fileLoc).glob('**/*.xlsx')
for path in pathlist:
    # because path is object not string
    obsDat = pd.read_excel(path, engine='openpyxl',sheet_name='Observed')
    newCols = []
    replace = False
    for c in obsDat.columns:
        if c in VariableRenames.keys():
            newCols.append(c.replace(c,VariableRenames[c]))
            replace = True
            if c == "Wheat.Leaf.Tips":
                print(str(path) + " tips")
        else:
            newCols.append(c)
    if replace == True:
        obsDat.columns = newCols
        with pd.ExcelWriter(path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer: 
            workbook = writer.book
            obsDat.to_excel(writer,index=False,sheet_name='Observed')
    
    try:
        obsDat = pd.read_excel(path, engine='openpyxl',sheet_name='MaxLeafSize')
        newCols = []
        replace = False
        for c in obsDat.columns:
            if c in MaxLeafSizeRenames.keys():
                newCols.append(c.replace(c,MaxLeafSizeRenames[c]))
                replace = True
            else:
                newCols.append(c)
        if replace == True:
            obsDat.columns = newCols
            with pd.ExcelWriter(path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer: 
                workbook = writer.book
                obsDat.to_excel(writer,index=False,sheet_name='MaxLeafSize')
    except:
        do = "Nothing"

C:\GitHubRepos\ApsimX\Tests\Validation\Wheat\data\Observed.xlsx tips


In [49]:
## Find and delete model of name and type
# def removeModel(Parent,modelName,modelType):
#     pos = 0
#     for c in Parent['Children']:
#         if (c['Name'] == modelName) and (c['$type'] == modelType):
#             del Parent['Children'][pos]
#             found = True
#             break
#         else:
#             removeModel(c,modelName,modelType)
#         pos += 1
        
# MasterFile = 'C:\GitHubRepos\ApsimX\Tests\Validation\Wheat\Wheat.apsimx'
# with open(MasterFile,'r') as MasterJSON:
#     Master = json.load(MasterJSON)
#     MasterJSON.close()
        
# removeModel(Master,"SetCropParams","Models.Manager, Models")
# removeModel(Master,"SowingReport","Models.Report, Models")

# with open(MasterFile,'w') as MasterJSON:
#     json.dump(Master ,MasterJSON,indent=2)